In [6]:
import pandas as pd
import numpy as np
import pathlib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
from sklearn.decomposition import NMF
import warnings
warnings.filterwarnings("ignore")
from sklearn.mixture import GaussianMixture
import os
import ot
import pickle
import argparse
import Levenshtein
import itertools
from helper_functions import *
##### input args

PROJECT = "gs-mrd"
release_version = "20240925"
data_version = "20240924"

files = [item for item in pathlib.Path(os.path.join("all_samples", data_version)).glob("*.csv")]
featuredf = dict()
for file in files:
    feature_name = file.name.split(".csv")[0]
    featuredf[feature_name] = pd.read_csv(file)

metadata = pd.read_excel("Metadata_TN_TP_MRDGW.xlsx")
metadata.groupby("Cancer")["SampleID"].count()

KeyError: 'Column not found: SampleID'

In [8]:
featuredf["EM"]

,SampleID,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,ZMB001D,0.011001,0.004247,0.006428,0.007153,0.004615,0.002236,0.000378,0.002802,0.003937,...,0.000094,0.001686,0.001478,0.000634,0.000654,0.001374,0.003584,0.002378,0.001691,0.005466
1,ZMB002C,0.010187,0.004176,0.006131,0.006600,0.004516,0.002253,0.000350,0.002593,0.003498,...,0.000082,0.001653,0.001484,0.000619,0.000570,0.001404,0.003749,0.002483,0.001712,0.005597
2,ZMB003D,0.010627,0.004042,0.006365,0.006709,0.004388,0.002076,0.000351,0.002576,0.003690,...,0.000107,0.001979,0.001843,0.000776,0.000834,0.001634,0.004123,0.002759,0.002022,0.006028
3,ZMB007D,0.010185,0.003943,0.006063,0.006551,0.004368,0.002101,0.000366,0.002638,0.003480,...,0.000127,0.002051,0.002250,0.000999,0.001039,0.001970,0.004420,0.003069,0.002388,0.006988
4,ZMB009B,0.010411,0.004064,0.006158,0.006728,0.004463,0.002153,0.000358,0.002628,0.003555,...,0.000091,0.001841,0.001483,0.000647,0.000638,0.001464,0.003850,0.002550,0.001755,0.005845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,QHAA44,0.010393,0.003899,0.006228,0.006563,0.004208,0.002017,0.000353,0.002545,0.003562,...,0.000108,0.002104,0.001814,0.000759,0.000868,0.001686,0.004103,0.002748,0.002003,0.006127
240,UHAA21,0.010459,0.003995,0.006138,0.006592,0.004367,0.002130,0.000369,0.002690,0.003611,...,0.000114,0.001995,0.001866,0.000798,0.000874,0.001653,0.004091,0.002776,0.002047,0.006152
241,MUGAAA20,0.010594,0.003989,0.006298,0.006632,0.004313,0.002086,0.000355,0.002520,0.003601,...,0.000111,0.001866,0.001880,0.000773,0.000836,0.001637,0.004114,0.002726,0.002024,0.006032
242,MUGAAA72,0.009755,0.003762,0.006097,0.006075,0.004124,0.002061,0.000358,0.002464,0.003562,...,0.000111,0.001983,0.001780,0.000761,0.000860,0.001612,0.003783,0.002600,0.001931,0.005591
